In [1]:
# Import dependencies
import json
from sqlalchemy import create_engine
import pandas as pd
import pymysql
pymysql.install_as_MySQLdb()
from datetime import datetime, date

In [2]:
# Import configuration variables
from config import sf_username, sf_password, sf_security_token
from config import remote_db_endpoint, remote_db_port
from config import remote_db_name, remote_db_user, remote_db_pwd

In [3]:
# Set up simple_salesforce
from simple_salesforce import Salesforce
sf = Salesforce(username=sf_username, password=sf_password, security_token=sf_security_token)

In [4]:
# Connect to your MySQL database
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

# Query 'student' table from MySql

In [5]:
# Query Data
student_data = pd.read_sql("SELECT * FROM student", conn)
student_data.head()

,ID_Student,StudentID,LastName,FirstName,MiddleName,BirthDate,Gender
0,33,25004961,Moore,Heather,Alice,None,F
1,34,25003514,Multak,Ilana,Cecille,None,F
2,35,25005833,Murillo,Jessica,Dorothy,None,F
3,36,25002589,Romanowski,Kandra,Genevieve,None,F
4,37,25007185,Hoffer,Katherine,Lynnette,None,F


In [6]:
# Transform Data to perfectly match between dataframe headers and field names in Salesforce
student_data = student_data.rename(columns={
                                        'ID_Student':'ID_Student__c',
                                        'StudentID':'StudentID__c',
                                        'LastName':'Last_Name__c',
                                        'FirstName':'Name',
                                        'MiddleName':'Middle__c',
                                        'BirthDate':'Birth_Date__c',
                                        'Gender':'Gender__c'
                                })
student_data.head()

,ID_Student__c,StudentID__c,Last_Name__c,Name,Middle__c,Birth_Date__c,Gender__c
0,33,25004961,Moore,Heather,Alice,None,F
1,34,25003514,Multak,Ilana,Cecille,None,F
2,35,25005833,Murillo,Jessica,Dorothy,None,F
3,36,25002589,Romanowski,Kandra,Genevieve,None,F
4,37,25007185,Hoffer,Katherine,Lynnette,None,F


In [7]:
# Change to readable form 
student_data['Birth_Date__c'] = student_data['Birth_Date__c'].apply(lambda x: x.strftime('%Y-%m-%d')) 
student_data

AttributeError: 'NoneType' object has no attribute 'strftime'

In [8]:
# Convert DataFrame to a list of dictionaries --> This can help facilitate a bulk insert
student_load = student_data.to_dict(orient='record')
student_load

[{'ID_Student__c': 33,
  'StudentID__c': '25004961',
  'Last_Name__c': 'Moore',
  'Name': 'Heather',
  'Middle__c': 'Alice',
  'Birth_Date__c': None,
  'Gender__c': 'F'},
 {'ID_Student__c': 34,
  'StudentID__c': '25003514',
  'Last_Name__c': 'Multak',
  'Name': 'Ilana',
  'Middle__c': 'Cecille',
  'Birth_Date__c': None,
  'Gender__c': 'F'},
 {'ID_Student__c': 35,
  'StudentID__c': '25005833',
  'Last_Name__c': 'Murillo',
  'Name': 'Jessica',
  'Middle__c': 'Dorothy',
  'Birth_Date__c': None,
  'Gender__c': 'F'},
 {'ID_Student__c': 36,
  'StudentID__c': '25002589',
  'Last_Name__c': 'Romanowski',
  'Name': 'Kandra',
  'Middle__c': 'Genevieve',
  'Birth_Date__c': None,
  'Gender__c': 'F'},
 {'ID_Student__c': 37,
  'StudentID__c': '25007185',
  'Last_Name__c': 'Hoffer',
  'Name': 'Katherine',
  'Middle__c': 'Lynnette',
  'Birth_Date__c': None,
  'Gender__c': 'F'},
 {'ID_Student__c': 38,
  'StudentID__c': '25006014',
  'Last_Name__c': 'Poocharoen',
  'Name': 'Pariya',
  'Middle__c': 'Marie

In [11]:
for row in student_load:

    record = {
        'ID_Student__c': row['ID_Student__c'],
        'StudentID__c': row['StudentID__c'],
        'Last_Name__c': row['Last_Name__c'],
        'Name':row['Name'],
        'Middle__c':row['Middle__c'],
        'Birth_Date__c':row['Birth_Date__c'],
        'Gender__c':row['Gender__c']
    }
    
    try:
        sf.Student__c.create(record)
    except Exception as e:
        print(e)

In [ ]:
# Inserting rows in bulk for Class table on Salesforce
'''
try:
    sf.bulk.Student__c.insert(student_push)
except Exception as e:
    print(e)
'''